In [200]:
#USPS_results replication
from sklearn import svm
import numpy as np
from tsvm import TSVM
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

with open ('usbs_databases', 'rb') as fp:
    usps_file = pickle.load(fp) 

partition = usps_file[0]
df_usps_sample = usps_file[1]
df_usps_test = usps_file[2]

def get_mean_and_std(partition,gram='0',method='svm'):
    sigma=6
    accuracy = np.zeros(50)
    gam = 1/(2*sigma**2)
    if(method=='cluster_kernel'): #cluster kernel
        trn = df_usps_sample.iloc[:,1:].to_numpy()
        tes = df_usps_test.iloc[:,1:].to_numpy()
        data = np.concatenate((trn,tes),axis=0)
        print('data dimensions', data.shape)
        lin_ker = extension_cluster_kernel(data,'linear')
        eig = lin_ker.eigvalues
        print('eig:',eig)
        print('eig length:',eig.shape)
        cut_off = k_th_largest_eig(eig,15)
        print('cut_off:',cut_off)
        lin_ker.poly_step([cut_off,1/2,1])
        for i in range(50):
            #for each i, use partition[i] as train, and the rest as test
            data_partition = partition[i].iloc[:,1:]
            targets = partition[i].iloc[:,0].to_numpy()
            inputs =  data_partition.to_numpy()
            unlabeled_df = df_usps_sample.drop(partition[i].index) #'drop' simply ingores the indeces from particion [i], giving us the 'rest'
            unlabeled_targets=unlabeled_df.iloc[:,0].to_numpy()
            unlabeled_inputs =unlabeled_df.iloc[:,1:].to_numpy()
            test_df = df_usps_test
            test_inputs = test_df.iloc[:,1:].to_numpy()
            test_targets= test_df.iloc[:,0].to_numpy()
            clf = svm.SVC(kernel=lin_ker.distance, C=100,class_weight='balanced')
            print('number of +1:',targets[targets==1].shape)
            print('number of -1:',targets[targets==-1].shape)
            clf.fit(inputs,targets)
            accuracy[i] = clf.score(test_inputs,test_targets)
            print('accuracy at iteration'+str(i)+':',accuracy[i]) 
        print(accuracy)
        print('mean:', np.mean(accuracy))
        print('std:', np.std(accuracy))
    elif(method=='svm'): #svm
        for i in range(50):
            #for each i, use partition[i] as train, and the rest as test
            data_partition = partition[i].iloc[:,1:]
            targets = partition[i].iloc[:,0].to_numpy()
            #data_partition = df_usps_sample.iloc[:,1:]
            #targets = df_usps_sample.iloc[:,0].to_numpy()
            inputs =  data_partition.to_numpy()
            test_df = df_usps_test
            test_targets=test_df.iloc[:,0].to_numpy()
            test_inputs =test_df.iloc[:,1:].to_numpy()
            #print(inputs_scaled.std(axis=0))
            #pca = PCA(n_components = 0.95, svd_solver='full')
            #pca.fit(inputs)
            #transformed_inputs = pca.transform(inputs)
            #transformed_test_inputs = pca.transform(test_inputs)
            clf = svm.SVC(kernel='rbf', C=100,gamma=gam,class_weight='balanced')
            clf.fit(inputs, targets)
            accuracy[i] = clf.score(test_inputs,test_targets)
            print('accuracy at iteration'+str(i)+':',accuracy[i])
        print('sigma:', sigma)
        print(accuracy)
        print('mean:', np.mean(accuracy))
        print('std:', np.std(accuracy))
    else:#tsvm

        for i in range(50):
            #for each i, use partition[i] as train, and the rest as test
            data_partition = partition[i].iloc[:,1:]
            targets = partition[i].iloc[:,0].to_numpy()
            inputs =  data_partition.to_numpy()
            unlabeled_df = df_usps_sample.drop(partition[i].index) #'drop' simply ingores the indeces from particion [i], giving us the 'rest'
            unlabeled_targets=unlabeled_df.iloc[:,0].to_numpy()
            unlabeled_inputs =unlabeled_df.iloc[:,1:].to_numpy()
            test_df = df_usps_test
            test_inputs = test_df.iloc[:,1:].to_numpy()
            test_targets= test_df.iloc[:,0].to_numpy()
            clf = TSVM()
            gamma = gam 
            C = 10
            clf.initial('rbf',gam,C)
            print(targets)
            clf.train(inputs,targets,unlabeled_inputs)
            test_predict = clf.predict(test_inputs)
            accuracy[i] = test_predict[test_predict==test_targets].size/test_predict.size
            print(accuracy[i])
            #clf = svm.SVC(kernel='rbf', C=10,gamma=gam,class_weight='balanced')
            #clf.fit(inputs, targets)
            #accuracy[i] = clf.score(test_inputs,test_targets)
        print(accuracy)
        print('mean:', np.mean(accuracy))
        print('std:', np.std(accuracy))


In [202]:
get_mean_and_std(partition,method='cluster_kernel')

data dimensions (4007, 256)
eig: [2.67833574e-07 3.36558527e-07 3.69995675e-07 ... 4.29565124e-01
 5.53495356e-01 1.00000000e+00]
eig length: (4007,)
cut_off: 0.11513246177898318
number of +1: (20,)
number of -1: (20,)
accuracy at iteration0: 0.8535127055306427
number of +1: (20,)
number of -1: (20,)
accuracy at iteration1: 0.8001993024414549
number of +1: (20,)
number of -1: (20,)
accuracy at iteration2: 0.8256103637269556
number of +1: (20,)
number of -1: (20,)
accuracy at iteration3: 0.8365719980069756
number of +1: (20,)
number of -1: (20,)
accuracy at iteration4: 0.8584952665670155
number of +1: (20,)
number of -1: (20,)
accuracy at iteration5: 0.817140009965122
number of +1: (20,)
number of -1: (20,)
accuracy at iteration6: 0.8365719980069756
number of +1: (20,)
number of -1: (20,)
accuracy at iteration7: 0.8315894369706028
number of +1: (20,)
number of -1: (20,)
accuracy at iteration8: 0.8295964125560538
number of +1: (20,)
number of -1: (20,)
accuracy at iteration9: 0.798704534

In [149]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
import sys
class extension_cluster_kernel:
    def __init__(self, data, type, parameters=None):
        self.dict = {}
        for i,d in enumerate(data):
            self.dict[str(d)] = i
        sigma=6
        gam = 1/(2*sigma**2)
        K = rbf_kernel(data,gamma = gam)
        D = np.zeros((K.shape[0],K.shape[1]))
        diagonalElements = np.sum(K,axis = 1)
        for i in range(D.shape[0]):
            D[i,i] = diagonalElements[i]**(-0.5)
        L = (D.dot(K)).dot(D)
        self.L = L
        self.eigvalues,self.eigvectors = np.linalg.eigh(L)
        if parameters:
            self.K = getattr(self, type)(parameters)
        else:
            self.K = getattr(self, type)()
    def compute_K(self,L):
        D_hat = np.diag((1./(np.diagonal(L) + sys.float_info.epsilon))**(0.5))
        K_hat = (D_hat.dot(L)).dot(D_hat)
        return K_hat
    def linear(self):
        return self.compute_K(self.L)
    def step(self,lambda_cut):
        tmp_eig = np.zeros((len(self.eigvalues),len(self.eigvalues)))
        for i,e in enumerate(self.eigvalues):
            if e >= lambda_cut:
                tmp_eig[i,i] = 1
            else:
                tmp_eig[i,i] = 0
        L_hat = (self.eigvectors.dot(tmp_eig)).dot(self.eigvectors.T)
        self.K = self.compute_K(L_hat)
        return self.K
    def linear_step(self,r):
        tmp_eig = np.diag(self.eigvalues)
        for i,e in enumerate(self.eigvalues):
            if e < r:
                tmp_eig[i,i] = 0
        L_hat = (self.eigvectors.dot(tmp_eig)).dot(self.eigvectors.T)
        self.K = self.compute_K(L_hat)
        return self.K
    def polynomial(self,t):
        tmp_eig = np.diag(np.power(self.eigvalues,t))
        L_hat = (self.eigvectors.dot(tmp_eig)).dot(self.eigvectors.T)
        self.K = self.compute_K(L_hat)
        return self.K
    def poly_step(self,parameter_list):
        cut_off = parameter_list[0]
        p = parameter_list[1]
        q = parameter_list[2]
        tmp_eig = np.zeros((len(self.eigvalues),len(self.eigvalues)))
        for i,e in enumerate(self.eigvalues):
            if e>=cut_off:
                tmp_eig[i,i] = np.power(e,p)
            else:
                tmp_eig[i,i] = np.power(e,q)
        L_hat = (self.eigvectors.dot(tmp_eig)).dot(self.eigvectors.T)
        self.K = self.compute_K(L_hat)
        return self.K
    
    def distance(self,X1,X2):
            gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
            index2list = np.zeros((X2.shape[0]),dtype=int)
            for j, x2 in enumerate(X2):
                index2list[j] = self.dict[str(x2)]
            for i, x1 in enumerate(X1):
                ind1 = self.dict[str(x1)]
                gram_matrix[i, :] = self.K[ind1,index2list]
            return gram_matrix

In [58]:
data = df_usps_sample.iloc[:,1:].to_numpy()
targets = df_usps_sample.iloc[:,0].to_numpy()
print(data.shape)
print(targets.shape)
my_kernel = extension_cluster_kernel(data,'linear_step',1)

(2000, 256)
(2000,)


In [59]:
print(my_kernel.K)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
from sklearn import svm

index = np.random.permutation(data.shape[0])

train_data = data[index[:40]]
Y = targets[index[:40]]

clf = svm.SVC(kernel = my_kernel.distance, C=100)
#clf = svm.SVC(kernel = 'precomputed')

#print(Y)
#print(train_data)

clf.fit(train_data,Y)
#clf.fit(my_kernel.distance(train_data,train_data),Y)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel=<bound method extension_cluster_kernel.distance of <__main__.extension_cluster_kernel object at 0x0000026487D84898>>,
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [76]:
test_data = data[index[40:2000]]
y_test = targets[index[40:2000]]
print(test_data.shape)
print(y_test.shape)
#clf.score(test_data,y_test)

(1960, 256)
(1960,)


In [100]:
pred = clf.predict(my_kernel.distance(train_data,test_data))


In [101]:
print(pred[pred==y_test].size/pred.size)

0.475


In [103]:
print(pred[pred==y_test])
print(pred)

[-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1]


In [62]:
def k_th_largest_eig(eig,k):
    arr = eig.copy()
    arr.sort() 
    return arr[-k]

ex = [1,2,3,4,5,6,7]
print(k_th_largest_eig(ex,3))

5


In [85]:
lin_ker = extension_cluster_kernel(data,'poly_step',[cut_off,1/2,2])

In [86]:
eigenv = lin_ker.eigvalues
print(eigenv)

[5.12272936e-06 5.36717222e-06 5.70439497e-06 ... 6.36924305e-01
 7.44778433e-01 1.00000000e+00]


In [87]:
cut_off = k_th_largest_eig(eigenv,15)
print(cut_off)

0.24595535248034162


In [80]:
lin_ker.linear_step(cut_off)

array([[ 1.        ,  0.16820461,  0.07673395, ...,  0.0084723 ,
         0.00751655,  0.02843456],
       [ 0.16820461,  1.        ,  0.88072997, ...,  0.59363339,
         0.25046989,  0.61090089],
       [ 0.07673395,  0.88072997,  1.        , ...,  0.46465528,
         0.19522036,  0.30295858],
       ...,
       [ 0.0084723 ,  0.59363339,  0.46465528, ...,  1.        ,
         0.71298524,  0.32859715],
       [ 0.00751655,  0.25046989,  0.19522036, ...,  0.71298524,
         1.        , -0.06475393],
       [ 0.02843456,  0.61090089,  0.30295858, ...,  0.32859715,
        -0.06475393,  1.        ]])

In [94]:
clf = svm.SVC(kernel = lin_ker.distance, C = 1000)
clf.fit(train_data,Y)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel=<bound method extension_cluster_kernel.distance of <__main__.extension_cluster_kernel object at 0x0000026488063828>>,
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [95]:
clf.score(test_data,y_test)

0.8112244897959183

In [159]:
lin_ker.poly_step([cut_off,1/2,2])


array([[ 1.        ,  0.00119113,  0.03364855, ..., -0.00997191,
        -0.02174076, -0.01350418],
       [ 0.00119113,  1.        ,  0.1156237 , ...,  0.17011473,
         0.08906751,  0.11379243],
       [ 0.03364855,  0.1156237 ,  1.        , ..., -0.05278648,
         0.03754403, -0.1303005 ],
       ...,
       [-0.00997191,  0.17011473, -0.05278648, ...,  1.        ,
         0.18297684,  0.54376966],
       [-0.02174076,  0.08906751,  0.03754403, ...,  0.18297684,
         1.        ,  0.14113327],
       [-0.01350418,  0.11379243, -0.1303005 , ...,  0.54376966,
         0.14113327,  1.        ]])